# LCEL Runnable 프로토콜

## 1. Runnable 프로토콜 소개

### Runnable이란?
- **Runnable**은 LangChain의 핵심 **인터페이스**
- 호출(invoke), 배치(batch), 스트리밍(stream) 등이 가능한 표준 인터페이스입니다
- LangChain의 모든 주요 구성요소(LLM, 프롬프트, 출력 파서, 리트리버 등)는 Runnable 프로토콜을 구현합니다

## 2. Runnable의 주요 메서드

### 핵심 메서드들
| 메서드 | 설명 | 사용 예시 |
|--------|------|-----------|
| `invoke()` | 단일 입력을 받아 출력 생성 | `runnable.invoke(input)` |
| `batch()` | 여러 입력을 병렬로 처리 | `runnable.batch([input1, input2])` |
| `stream()` | 출력을 스트리밍으로 생성 | `runnable.stream(input)` |
| `astream_events()` | 이벤트 스트리밍 (고급) | `runnable.astream_events(input)` |

### 입력/출력 타입 (컴포넌트별)
| 컴포넌트 | 입력 타입 | 출력 타입 |
|----------|-----------|-----------|
| **Prompt** | 딕셔너리 객체 | PromptValue |
| **ChatModel** | 문자열, 메시지 리스트, PromptValue | ChatMessage |
| **LLM** | 문자열, 메시지 리스트, PromptValue | 문자열 |
| **OutputParser** | LLM 또는 ChatModel의 출력 | 파서에 따라 다름 |
| **Retriever** | 문자열 | Document 리스트 |
| **Tool** | 문자열 또는 객체 | 도구에 따라 다름 |


**실습 준비 - 기본 설정**

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.5)

### 기본 Runnable 실습

**invoke() 메서드 - 단일 입력 처리**

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 템플릿 생성 (Runnable)
prompt = PromptTemplate.from_template("다음 주제에 대해 한줄로 간단히 설명해주세요: {topic}")

# 2. 출력 파서 생성 (Runnable)
output_parser = StrOutputParser()

# invoke 메서드를 사용하여 각 단계별로 실행
input_data = {"topic": "랭체인"} # 딕셔너리

In [ ]:
# 1단계: 프롬프트 생성
formatted_prompt = prompt.invoke(input_data)

print(formatted_prompt)

In [ ]:
# 2단계: LLM 실행
llm_response = llm.invoke(formatted_prompt)
print(llm_response)

In [ ]:
# 3단계: 출력 파싱
final_output = output_parser.invoke(llm_response)
print(final_output)

**체인 구성 - Pipe 연산자 (|) 사용**
- LCEL의 핵심 기능: Pipe 연산자(`|`)를 사용한 체인 구성
- 여러 Runnable을 연결하여 하나의 체인으로 만들 수 있습니다


In [ ]:
# 방법 1: Pipe 연산자 사용
chain = prompt | llm | output_parser

# 체인을 하나의 Runnable로 사용
result = chain.invoke({"topic": "경남대학교"})
print(result)